In [6]:

import requests
import time
import numpy as np
import pandas as pd
import json

In [2]:
string = 'http://api.sportradar.us/nfl-t1/2015/REG/5/WAS/ATL/injuries.json?api_key=jdmq6nfne2a7ucg6g5jkvsz7'
answer = requests.get(string)
test  = answer.json()
test.keys()

test.keys()

[u'scheduled', u'status', u'home_team', u'away_team', u'id']

In [3]:

def flatten_injuries(dict_in):
    i=0
    n=0
    m=0
    list_dicts = []
    
    for key1 in dict_in.keys():
        if (type(dict_in[key1]) == dict):
            for key2 in dict_in[key1].keys():
                #print key2
                if (type(dict_in[key1][key2]) == list):
                    for i in range(0,len(dict_in[key1][key2])):
                        #print dict_in[key1][key2][i]
                        #df_out = pd.DataFrame(dict_in[key1][key2][i])
                        if (type(dict_in[key1][key2][i]) == dict):
                            for key3 in dict_in[key1][key2][i].keys():
                                #print key3
                                if (type(dict_in[key1][key2][i][key3]) == list):
                                    #print dict_in[key1][key2][i][key3]
                                    for n in range(0,len(dict_in[key1][key2][i][key3])):
                                        if (type(dict_in[key1][key2][i][key3][n]) == dict):
                                            #print type(dict_in[key1][key2][i][key3][n])
                                            dict1 = dict_in[key1][key2][i][key3][n].copy()
                                            #print 'dict 1 is', dict1
                                            dict2 = dict_in[key1][key2][i].copy()
                                            #print 'dict 2 is '
                                            del dict2[key3]
                                            row = dict(dict2, **dict1)
                                            #print 'row is', row
                                            dict3 = dict_in[key1].copy()
                                            #print dict3
                                            del dict3[key2]
                                            list_dicts.append(dict(dict3, **row))
                                            m += 1
                                            
    df_out = pd.DataFrame(list_dicts)
    game_id = pd.Series([dict_in['id']]*len(df.index), name = 'game_id')
    df_out = pd.concat([series,df], axis=1)
    return df_out

In [4]:
series = pd.Series([test['id']]*len(df.index), name = 'game_id')

series

NameError: name 'df' is not defined

In [ ]:
pd.concat([series,df], axis=1)

In [ ]:
df = flatten_injuries(test)

In [ ]:
df

In [ ]:
b = dict1.copy()

In [ ]:
del b['b']

In [ ]:
type(b)

In [ ]:
dict1

In [5]:
type(dict1)

NameError: name 'dict1' is not defined

In [6]:
b

NameError: name 'b' is not defined

# Gathering game statistics

In [7]:
string = 'http://api.sportradar.us/nfl-t1/2015/REG/1/MIN/SF/statistics.json?api_key=jdmq6nfne2a7ucg6g5jkvsz7'
answer = requests.get(string)
test  = answer.json()
test.keys()

[u'scheduled', u'status', u'home_team', u'away_team', u'id']

In [8]:
test['home_team']['statistics']['punting']

{u'players': [{u'avg': 46.667,
   u'avg_ret': 4.5,
   u'blk': 0,
   u'id': u'9000be32-15ad-4c43-bf8d-79a9c7113cdd',
   u'in20': 2,
   u'in20_pct': 66.667,
   u'jersey': 5,
   u'lg': 52,
   u'name': u'Bradley Pinion',
   u'net_avg': 43.667,
   u'net_yds': 131,
   u'position': u'P',
   u'punts': 3,
   u'ret': 2,
   u'ret_yds': 9,
   u'sfty': 0,
   u'tb': 0,
   u'tb_pct': 0.0,
   u'yds': 140}],
 u'team': {u'avg': 46.667,
  u'avg_ret': 4.5,
  u'blk': 0,
  u'in20': 2,
  u'in20_pct': 66.667,
  u'lg': 52,
  u'net_avg': 43.667,
  u'net_yds': 131,
  u'punts': 3,
  u'ret': 2,
  u'ret_yds': 9,
  u'sfty': 0,
  u'tb': 0,
  u'tb_pct': 0.0,
  u'yds': 140}}

In [52]:
def flatten_game_stats(dict_in, stat_key):
    i=0
    n=0
    m=0
    player_stats = []
    team_stats = []
    
    for key1 in dict_in.keys():
        if (type(dict_in[key1]) == dict):
            for key2 in dict_in[key1].keys():
                if (type(dict_in[key1][key2]) == dict):
                    for key3 in dict_in[key1][key2][stat_key].keys():
                        if (key3 == 'players'):
                            for n in range(0,len(dict_in[key1][key2][stat_key][key3])):
                                player_row = dict_in[key1][key2][stat_key][key3][n].copy()
                                player_row['market'] = dict_in[key1]['market']
                                player_row['name'] = dict_in[key1]['name']
                                player_stats.append(player_row)
                                n+=1
                        elif (key3 == 'team'):
                            team_row = dict_in[key1][key2][stat_key][key3]
                            team_row['market'] = dict_in[key1]['market']
                            team_row['name'] = dict_in[key1]['name']
                            team_stats.append(team_row)
                        
    player_df_out = pd.DataFrame(player_stats)
    team_df_out = pd.DataFrame(team_stats)
    
    game_id_players = pd.Series([dict_in['id']]*len(player_df_out.index), name = 'game_id')
    game_id_team = pd.Series([dict_in['id']]*len(team_df_out.index), name = 'game_id')
    
    scheduled_players = pd.Series([dict_in['scheduled']]*len(player_df_out.index), name = 'scheduled')
    scheduled_team = pd.Series([dict_in['scheduled']]*len(team_df_out.index), name = 'scheduled')
    
    player_df_out = pd.concat([game_id_players, scheduled_players, player_df_out], axis=1)
    team_df_out = pd.concat([game_id_team, scheduled_team, team_df_out], axis = 1)
    
    return player_df_out,team_df_out

In [53]:
from pandas.io.json import json_normalize

In [54]:
test['home_team']['name']

u'49ers'

In [55]:
x,y = flatten_game_stats(test, stat_key = 'punting')

In [56]:
x.columns

Index([  u'game_id', u'scheduled',       u'avg',   u'avg_ret',       u'blk',
              u'id',      u'in20',  u'in20_pct',    u'jersey',        u'lg',
          u'market',      u'name',   u'net_avg',   u'net_yds',  u'position',
           u'punts',       u'ret',   u'ret_yds',      u'sfty',        u'tb',
          u'tb_pct',       u'yds'],
      dtype='object')

In [60]:
y.columns

Index([  u'game_id', u'scheduled',       u'avg',   u'avg_ret',       u'blk',
            u'in20',  u'in20_pct',        u'lg',    u'market',      u'name',
         u'net_avg',   u'net_yds',     u'punts',       u'ret',   u'ret_yds',
            u'sfty',        u'tb',    u'tb_pct',       u'yds'],
      dtype='object')

In [61]:
y.head()

,game_id,scheduled,avg,avg_ret,blk,in20,in20_pct,lg,market,name,net_avg,net_yds,punts,ret,ret_yds,sfty,tb,tb_pct,yds
0,718a4f52-c6af-4080-b955-95e8769b68a7,2015-09-15T02:20:00+00:00,46.667,4.5,0,2,66.667,52,San Francisco,49ers,43.667,131,3,2,9,0,0,0,140
1,718a4f52-c6af-4080-b955-95e8769b68a7,2015-09-15T02:20:00+00:00,37.500,0.0,0,2,50.000,44,Minnesota,Vikings,37.500,150,4,2,0,0,0,0,150


In [9]:
type(y.scheduled[1])

NameError: name 'y' is not defined

# Import JSON function

In [15]:
def api_get_injuries(year,season_type,week,home_team,away_team):
    string = 'http://api.sportradar.us/nfl-t1/%i/%s/%i/%s/%s/statistics.json?api_key=jdmq6nfne2a7ucg6g5jkvsz7' %(year, season_type, week, home_team, away_team)
    print string
    answer = requests.get(string)
    dict_out  = answer.json()
    return dict_out
    
    


In [19]:
api_get_injuries(2015,u'REG',5,u'WAS',u'ATL')

http://api.sportradar.us/nfl-t1/2015/REG/5/WAS/ATL/statistics.json?api_key=jdmq6nfne2a7ucg6g5jkvsz7


{u'away_team': {u'id': u'WAS',
  u'market': u'Washington',
  u'name': u'Redskins',
  u'points': 19,
  u'remaining_challenges': 2,
  u'remaining_timeouts': 2,
  u'statistics': {u'defense': {u'players': [{u'ast': 0,
      u'bk': 0,
      u'comb': 2,
      u'force_fum': 0,
      u'fum_rec': 0,
      u'fum_td': 0,
      u'id': u'e84b685e-6b52-4214-b264-123ce0672021',
      u'int': 0,
      u'int_lg': 0,
      u'int_td': 0,
      u'int_yds': 0,
      u'jersey': 98,
      u'misc_ast': 0,
      u'misc_comb': 0,
      u'misc_force_fum': 0,
      u'misc_fum_rec': 0,
      u'misc_tackle': 0,
      u'name': u'Terrance Knighton',
      u'pd': 0,
      u'position': u'NT',
      u'qh': 0,
      u'sack': 0.0,
      u'sack_yds': 0.0,
      u'sfty': 0,
      u'sfty_1pt': 0,
      u'sp_ast': 0,
      u'sp_comb': 0,
      u'sp_force_fum': 0,
      u'sp_fum_rec': 0,
      u'sp_tackle': 0,
      u'tackle': 2,
      u'tlost': 0},
     {u'ast': 0,
      u'bk': 0,
      u'comb': 0,
      u'force_fum': 0,
    

In [14]:
type(2015)

int

# Getting the schedule parsed

In [156]:
string = 'http://api.sportradar.us/nfl-t1/2015/REG/5/schedule.json?api_key=jdmq6nfne2a7ucg6g5jkvsz7'
answer = requests.get(string)
test  = answer.json()
test.keys()

[u'bye_week', u'games', u'id', u'number']

In [159]:
test['games'][1]

{u'away': u'CHI',
 u'away_rotation': u'',
 u'broadcast': {u'network': u'FOX', u'satellite': u'707'},
 u'home': u'KC',
 u'home_rotation': u'',
 u'id': u'64aa74a7-9bf5-4dcb-99b9-5447de01b96d',
 u'scheduled': u'2015-10-11T17:00:00+00:00',
 u'status': u'closed',
 u'venue': {u'address': u'One Arrowhead Drive',
  u'capacity': 79451,
  u'city': u'Kansas City',
  u'country': u'USA',
  u'id': u'2ec4c411-dac2-403d-b091-6b6aa4a0a914',
  u'name': u'Arrowhead Stadium',
  u'state': u'MO',
  u'surface': u'turf',
  u'type': u'outdoor',
  u'zip': u'64129'},
 u'weather': {u'condition': u'Sunny',
  u'humidity': 53,
  u'temperature': 72,
  u'wind': {u'direction': u'SSW', u'speed': 11}}}

In [71]:
test['type']

u'REG'

In [128]:
def flatten_schedule(dict_in):
    n=0
    weeks = []
    key1 = 'games'
    for n in range(0,len(dict_in[key1])):
        week_row = dict_in[key1][n].copy()
        del week_row['venue']
        del week_row['weather']
        del week_row['away_rotation']
        del week_row['home_rotation']
        week_row['surface'] = dict_in[key1][n]['venue']['surface']
        week_row['temperature'] = dict_in[key1][n]['weather']['temperature']
        week_row['condition'] = dict_in[key1][n]['weather']['condition']
        week_row['week'] = dict_in['number']
        weeks.append(week_row)
    df_out = pd.DataFrame(weeks)
    return df_out


In [136]:
x.ix[0]

away                                            IND
broadcast                  {u'network': u'CBS/NFL'}
condition                                    Clear 
home                                            HOU
id             4d968b1d-0393-473a-a942-2345913cb5fe
scheduled                 2015-10-09T00:25:00+00:00
status                                       closed
surface                                        turf
temperature                                      82
week                                              5
Name: 0, dtype: object

In [169]:
from datetime import datetime
string = '2015-10-09T00:25:00'


ValueError: time data '2015-10-09T00:25:00' does not match format '%yyyy-%mm-%ddT%HH:%MM%SS'

# Import JSON injuries

In [170]:
def api_get_injuries(week_row):
    away = str(week_row['away'])
    home = str(week_row['home'])
    num = str(week_row['week'])
        nfl-t1/2015/REG/5/WAS/ATL/injuries.json?api_key=jdmq6nfne2a7ucg6g5jkvsz7'
    
    string = 'http://api.sportradar.us/nfl-t1/%s/%f/%s/%s/statistics.json?api_key=jdmq6nfne2a7ucg6g5jkvsz7' %(year, season, week, home_team, away_team)
    answer = requests.get(string)
    dict_out  = answer.json()
    return dict_out

SyntaxError: invalid syntax (<ipython-input-170-c613209714aa>, line 2)